## Sveska za importovanje i pravljenje jobova za CAS-VIT mrežu

CAS-ViT se zasniva na Visual Transformer arhitekturi, više informacija na linku.

https://paperswithcode.com/paper/cas-vit-convolutional-additive-self-attention

(samo sam sortirao po broju parametara i uzeo ovu random mrezu, nemam pojma da li je dobra, i rad za sada ne razumem)

Ova sveska uz prateće fajlove automatski skida weightove za CAS-ViT model zadate veličine, i posle taj model šalje na AIHUB za testiranje.

Za sada sam pokrenuo samo xs model, koji se izvršava super brzo i ima manje parametara od MobileNetV2, doduše mnogo više layera koji su raspoređeni na više COMPUTE unita, što je koliko ja razumem dobro.

Ne znam da li je dobro da modele čuvamo na GIT-u (Drakula neka kaže kako bi to trebalo), pa sam sve fajlove koje se generišu stavio u gitignore.


In [1]:
import gdown
import torch
import rcvit
import torchsummary
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)
from utils import helper, input_getter, qai_hub_jobs, tfhelper

import skimage as ski

/home/centar15-desktop1/LPCV_2025_T1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/centar15-desktop1/LPCV_2025_T1/.venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/centar15-desktop1/LPCV_2025_T1/.venv/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
2025-02-24 17:18:51.282238: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slight

In [2]:
# Model selection (options: "xs", "s", "m", "t")

modelID = "s"

driveFileIDs = dict()

driveFileIDs["xs"] = "16wKcwF6QMW5w_lyPYnDKjMNuoxQDfrLK"
driveFileIDs["s"]  = "1facFRq8s8oelYUtK1fj3fcfdoWoKDBQQ"
driveFileIDs["m"]  = "13sQpSEf0h_uuh0jRy9V0yIW6ZsbDpVGy"
driveFileIDs["t"]  = "1NqoIUPbwBC91RTjTUvubAbOfGqo1VYT0"

networks = dict()
networks["xs"] = rcvit.rcvit_xs
networks["s"]  = rcvit.rcvit_s
networks["m"]  = rcvit.rcvit_m
networks["t"]  = rcvit.rcvit_t

file_id = driveFileIDs[modelID]

modelName = "CASVIT_{}".format(modelID)

file_path = modelName + ".pth"

if not os.path.exists(file_path):
    gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)
    print("Model downloaded successfully.")
else:
    print(f"File {file_path} already exists.")

File CASVIT_s.pth already exists.


In [3]:
net: rcvit.RCViT = networks[modelID]()
print(net)

RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
            (3): GELU(approximate='none')
            (4): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(48, eps=1e-05, momentum=0.1, aff

In [4]:
#net.cuda()

#torchsummary.summary(net, (3, 224, 224))

In [4]:
# Loading pre-trained weights

weightDict = torch.load(file_path, map_location=torch.device("cpu"))
net.load_state_dict(weightDict["model"])

<All keys matched successfully>

In [6]:
torch.onnx.export(net, torch.randn((1, 3, 224, 224)), 'model.onnx')

In [14]:
# net = net.half()

# net = net.cuda()

net.eval()

net = net.half()

net = net.float()

input = input_getter.local_image_getter("/home/centar15-desktop1/LPCV_2025_T1/datasets/imagenet/coco_80/bowl/0000021345.jpg")
input_tensor = input.get_input_torch()
#input_tensor = input_tensor.half()
net(torch.tensor(input_tensor)).shape

/tmp/ipykernel_247069/3282206843.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  net(torch.tensor(input_tensor)).shape


torch.Size([1, 1000])

In [16]:
input.get_input_numpy().shape

(1, 3, 224, 224)

In [17]:
type(input.get_input_numpy())

numpy.ndarray

In [13]:
traced_model = torch.jit.trace(net, input_tensor)

import qai_hub as hub

cj = hub.submit_compile_job(
    model=traced_model,
    device=hub.Device("Snapdragon 8 Elite QRD"),
    input_specs=dict(image=input_tensor.cpu().numpy().shape),)

RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [18]:
# Kada se uradi ovi jobovi na linkovima se moze naci vizuelizacija mreze i informacije za dalju analizu

compile, profile, inference = qai_hub_jobs.compile_profile_inference_tensor(net, input_tensor)

Uploading tmpe3gnn88s.pt


100%|██████████| 23.5M/23.5M [00:01<00:00, 22.9MB/s]


Scheduled compile job (jpx96yy1p) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpx96yy1p/

Waiting for compile job (jpx96yy1p) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jpyzj667g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpyzj667g/



Uploading dataset: 294kB [00:00, 798kB/s]                            2.67MB/s]


Scheduled inference job (jp042ll6g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp042ll6g/



In [19]:
categories = helper.get_imagenet_categories()

# Rezultati dobijeni na cloudu

helper.inference_job_probabilities(inference)

tmp33ojkcks.h5: 100%|██████████| 14.5k/14.5k [00:00<00:00, 1.64MB/s]


Top-5 predictions for Cloud model on :
659 b'mixing bowl'        65.9%
969 b'eggnog'             25.8%
968 b'cup'                 3.0%
666 b'mortar'              1.7%
809 b'soup bowl'           1.0%


In [20]:
# Vraca verovatnoce lokalne PyTorch mreze

helper.print_probablities_from_output(net(input.get_input_torch()), categories=categories, modelname = 'Torch Local')

Top-5 predictions for Torch Local on :
659 b'mixing bowl'        73.3%
969 b'eggnog'             18.5%
968 b'cup'                 3.1%
666 b'mortar'              1.9%
809 b'soup bowl'           1.2%


In [21]:
# Vraca verovatnoce TensorFlow mreze

download_path = tfhelper.download_model_from_compile_job(compile, modelName)
TFModel = tfhelper.TFHelper(download_path)
TFModel.run_inference(input.get_input_numpy())
helper.print_probablities_from_output(TFModel.run_inference(input.get_input_numpy()), categories=categories, modelname = 'TensorFlow')

CASVIT_s.tflite: 100%|██████████| 22.1M/22.1M [00:07<00:00, 2.97MB/s]
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Downloaded model to CASVIT_s.tflite
Top-5 predictions for TensorFlow on :
659 b'mixing bowl'        73.3%
969 b'eggnog'             18.5%
968 b'cup'                 3.1%
666 b'mortar'              1.9%
809 b'soup bowl'           1.2%
